In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132590")
exp = Experiment(workspace=ws, name="udacity-project-hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132590
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-132590


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "newcompute12345"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
### YOUR CODE HERE ### 
parameter_space={
    "C": uniform(0,1),
    "max_iter" : choice(10,20,50,100,150)
}
ps =RandomParameterSampling(parameter_space)

# Specify a Policy
highpolicy = BanditPolicy(slack_factor=0.3)### YOUR CODE HERE ###

#if "training" not in os.listdir():
#    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
#from azureml.core.script_run_config import ScriptRunConfig
#est = ScriptRunConfig(source_directory="./", script= "train.py", compute_target=cpu_cluster )
est = SKLearn(source_directory="./", entry_script="train.py",compute_target=cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4, 
                                     policy=highpolicy, 
                                     estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [ ]:
#HDrun.get_best_run_by_primary_metric()

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
#from azureml.train.hyperdrive import HyperDriveRun
HDrun = exp.submit(hyperdrive_config)
HDrun.wait_for_completion(show_output=True)
#HPrun.get_metrics()
#RunDetails(HDrun).show()


In [ ]:
RunDetails(HDrun).show()

In [ ]:
HDrun.get_best_run_by_primary_metric().get_metrics()

In [ ]:
import joblib
# Get your best run and save the model from that run.
bestHDrun = HDrun.get_best_run_by_primary_metric()
bestHDmodel= bestHDrun.register_model(model_name="bestHDrun", model_path="./outputs/HyperdriveModel.joblib", tags=bestHDrun.get_metrics())
#joblib.dump(bestHDrun, 'HyperdriveModel.joblib')

 
### YOUR CODE HERE ###

In [ ]:
print(bestHDmodel)

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
webpath = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(webpath)


In [43]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data= pd.concat([x_train.reset_index(drop=True), y_train], axis=1)

In [45]:
from azureml.data.dataset_factory import TabularDatasetFactory

#filepath="~/cloudfiles/code/Users/odl_user_132590/traindata.csv"
train_data.to_csv("./outputs/traindata.csv")

datastore= ws.get_default_datastore()
datastore.upload(src_dir="./outputs",target_path="./outputs")

train_data = TabularDatasetFactory.from_delimited_files(datastore.path("./outputs/traindata.csv"))

Uploading an estimated of 1 files
Uploading ./outputs/traindata.csv
Uploaded ./outputs/traindata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [46]:
type(train_data)

azureml.data.tabular_dataset.TabularDataset

In [47]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

compute_name= os.environ.get("AML_COMPUTE_CLUSTER_NAME","cpu-cluster")
vmsize = os.environ.get("AML_COMPUTE_CLUSTER_SKU","STANDARD_D2_V2")
compute_minnodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES",0)
compute_maxnodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES",4)

if compute_name in ws.compute_targets:
    Compute_target = ws.compute_targets[compute_name]
    if Compute_target and type(Compute_target) is AmlCompute:
        print ("found compute target.. using : " + compute_name)
else:
    print("creating new compute target..")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vmsize, min_nodes=compute_minnodes, max_nodes=compute_maxnodes)
    Compute_target = ComputeTarget.create(ws,compute_name,provisioning_config)
    compute_target.wait_for_completion(timeout_in_minutes=20)

    print(compute_target.get_status().serialize())


found compute target.. using : cpu-cluster


In [48]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_settings={
    "experiment_timeout_minutes" : 15,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes" : 5,
    "enable_onnx_compatible_models" : True
}

automl_config = AutoMLConfig(
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=4,
    compute_target= Compute_target,
    **automl_settings)

In [49]:
# Submit your automl run
exp_aml = Experiment(workspace=ws, name="udacity-project-aml")
automl_run = exp_aml.submit(automl_config, show_output=True)
automl_run.wait_for_completion()

Running on remote.


{'runId': 'AutoML_1b5a0f08-ecc0-4558-a2a1-b60780d6ded0',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-31T08:36:24.112089Z',
 'endTimeUtc': '2020-12-31T09:01:22.79989Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project-aml","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-132590","workspace_name":"quick-starts-ws-132590","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":4,"y_min":null,"y_max":null,"num_classes":nu

In [53]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [55]:
best_run_aml.get_metrics()

{'precision_score_weighted': 0.7885269736945565,
 'balanced_accuracy': 0.5,
 'log_loss': 0.35100495300108037,
 'average_precision_score_weighted': 0.800416337999143,
 'precision_score_micro': 0.8879778945461134,
 'recall_score_macro': 0.5,
 'average_precision_score_micro': 0.8602505781498233,
 'weighted_accuracy': 0.9842940505175154,
 'recall_score_micro': 0.8879778945461134,
 'accuracy': 0.8879778945461134,
 'f1_score_macro': 0.47032947335692266,
 'AUC_macro': 0.49944576699190113,
 'average_precision_score_macro': 0.501175734021351,
 'f1_score_weighted': 0.8352968423783815,
 'f1_score_micro': 0.8879778945461134,
 'AUC_weighted': 0.4994457669919012,
 'matthews_correlation': 0.0,
 'recall_score_weighted': 0.8879778945461134,
 'norm_macro_recall': 0.0,
 'precision_score_macro': 0.4439889472730567,
 'AUC_micro': 0.8878496088608031,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_1b5a0f08-ecc0-4558-a2a1-b60780d6ded0_8/confusion_matrix',
 'accuracy_table': 'aml://artifactId

In [57]:
# Retrieve and save your best automl model.
### YOUR CODE HERE ###
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_aml_model, onnx_model = automl_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")
best_run_aml = automl_run.get_best_child()

#joblib.dump(best_run_aml, 'amlRun.joblib') 